In [ ]:
import os, json, math, time, random
from pathlib import Path
from typing import List, Dict, Any

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from math import ceil
from tqdm.auto import tqdm
import KEYS

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Métricas
from pathlib import Path
import sys
sys.path.insert(0, str(Path.cwd().parent))

from metricas.metrics_client import getRelevance, getFactuality, getReadability

# Reproducibilidad
def set_seed(seed: int):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Prompts (usa los mismos que en entrenamiento)
SYSTEM_PROMPT = (
    "You simplify clinical trial protocol text into a plain-language summary for the general public. "
    "Keep to 6–8th grade readability, avoid diagnoses and speculation, no hallucinations, "
    "and preserve key facts (objective, population, interventions, outcomes, timelines, safety)."
)
USER_PREFIX = "Using the following clinical trial protocol text as input, create a plain language summary.\n\n"

## Función para cargar el modelo

In [ ]:
def load_model_and_tokenizer(model_dir: str, device: str = DEVICE):
    model_dir = str(Path(model_dir).resolve())
    cfg_path = Path(model_dir) / "adapter_config.json"
    if not cfg_path.exists():
        raise FileNotFoundError(f"No existe {cfg_path}")

    with open(cfg_path, "r", encoding="utf-8") as f:
        adapter_cfg = json.load(f)
    base = adapter_cfg.get("base_model_name_or_path")
    if not base:
        raise ValueError("adapter_config.json no contiene 'base_model_name_or_path'.")

    # Tokenizer desde el directorio del adapter (incluye el token extra)
    tok = AutoTokenizer.from_pretrained(model_dir, use_fast=True, trust_remote_code=True)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    tok.padding_side = "left"

    # Modelo base
    base_model = AutoModelForCausalLM.from_pretrained(
        base,
        torch_dtype=torch.float16 if device.startswith("cuda") else torch.float32,
        trust_remote_code=True
    ).to(device)

    #  alinear tamaño de vocab del base al del tokenizer usado en el adapter
    base_model.resize_token_embeddings(len(tok))

    # Cargar adapters LoRA
    model = PeftModel.from_pretrained(base_model, model_dir)
    model.eval()
    model.config.pad_token_id = tok.pad_token_id

    # token de fin de oración personalizado
    eos_id = None
    try:
        eid = tok.convert_tokens_to_ids("<|sentence_end|>")
        if eid is not None and eid != tok.unk_token_id:
            eos_id = eid
    except Exception:
        pass

    return model, tok, eos_id

## Parámetros del proceso de evaluación

In [ ]:
# === Parámetros de evaluación ===
MODEL_DIR = "ollama/outputs/meta-llama__Llama-3.2-3B-Instruct-6_epocas/final"
DATASET_CSV = "../data/pls_abstract_pairs_with_metrics.csv"
SPLIT = "test"                  # 'test' o 'train'
NUM_EXAMPLES = 3               # cuántos ejemplos del split
SAMPLES_PER_EXAMPLE = 3         # cuántos resúmenes por ejemplo

SEED = 99
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# generación 
GEN_CFG = dict(
    max_new_tokens= 1024,
    do_sample=True,
    temperature=0.2,
    top_p=0.9,
    no_repeat_ngram_size=6,
    repetition_penalty=1.15,
)

# batches
BATCH_GEN = 2           # prompts por batch para generate()
BATCH_METRICS = 4       # micro-lote por POST a la API de métricas

# servicio de métricas
METRICS_URL = "http://127.0.0.1:8000" # corriendo en local

# salida
OUT_DIR = "model_evals/llama3.2-3b" 
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

set_seed(SEED)

## Cargar el modelo y los datos

In [ ]:
model, tokenizer, EOS_ID = load_model_and_tokenizer(MODEL_DIR, DEVICE)
print("EOS_ID (custom):", EOS_ID)

In [ ]:
df = pd.read_csv(DATASET_CSV)
cols = {"source_text","target_text","split"}
if not cols.issubset(df.columns):
    raise ValueError(f"El CSV debe contener {cols}")

val_df = df[df["split"] == SPLIT].dropna(subset=["source_text","target_text"]).reset_index(drop=True)
if len(val_df) == 0:
    raise ValueError(f"No hay filas para split='{SPLIT}'")
if NUM_EXAMPLES < len(val_df):
    val_df = val_df.sample(n=NUM_EXAMPLES, random_state=SEED).reset_index(drop=True)

sources = val_df["source_text"].tolist()
targets = val_df["target_text"].tolist()

def build_prompt(src: str) -> str:
    return tokenizer.apply_chat_template(
        [{"role":"system","content":SYSTEM_PROMPT},
         {"role":"user","content":USER_PREFIX + str(src)}],
        tokenize=False, add_generation_prompt=True
    )
prompts = [build_prompt(s) for s in sources]
len(prompts), len(targets)

## Generación

In [ ]:
@torch.no_grad()
def generate_samples(prompts: List[str], n_samples: int) -> List[List[str]]:
    """
    Devuelve una lista length=len(prompts). Cada elemento es una lista de 'n_samples' resúmenes.
    Sin manipular semillas. Muestra progreso total (rondas × batches).
    """
    all_outputs = [list() for _ in range(len(prompts))]
    base_cfg = GEN_CFG.copy()
    if EOS_ID is not None:
        base_cfg["eos_token_id"] = EOS_ID
    base_cfg["pad_token_id"] = tokenizer.pad_token_id

    total_batches = n_samples * ceil(len(prompts) / max(1, BATCH_GEN))
    pbar = tqdm(total=total_batches, desc="Generando", unit="batch")

    for _ in range(n_samples):
        for i in range(0, len(prompts), BATCH_GEN):
            j = min(i + BATCH_GEN, len(prompts))
            batch_prompts = prompts[i:j]
            inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True).to(DEVICE)
            gen = model.generate(**inputs, **base_cfg)
            cut = inputs["input_ids"].shape[1]
            outs = tokenizer.batch_decode(gen[:, cut:], skip_special_tokens=True)
            for idx, txt in enumerate(outs):
                all_outputs[i + idx].append(txt.strip())
            pbar.update(1)

    pbar.close()
    return all_outputs

generated = generate_samples(prompts, SAMPLES_PER_EXAMPLE)
assert len(generated) == len(prompts)
assert all(len(xs) == SAMPLES_PER_EXAMPLE for xs in generated)

## Métricas

In [ ]:
def chunked(iterable, n):
    for i in range(0, len(iterable), n):
        yield iterable[i:i+n]

rows = []  # filas por ejemplo×muestra
for doc_id, (src, tgt, gens) in enumerate(zip(sources, targets, generated)):
    # evaluamos por micro-lotes contra la API
    pairs_src, pairs_gen = [], []
    for sidx, gen_txt in enumerate(gens):
        pairs_src.append(src)
        pairs_gen.append(gen_txt)

    # relevance
    rel_scores = []
    for c_src, c_gen in zip(chunked(pairs_src, BATCH_METRICS), chunked(pairs_gen, BATCH_METRICS)):
        rel_scores.extend(getRelevance(c_src, c_gen, base_url=METRICS_URL, timeout=180.0))

    # factuality
    fac_scores = []
    for c_src, c_gen in zip(chunked(pairs_src, BATCH_METRICS), chunked(pairs_gen, BATCH_METRICS)):
        fac_scores.extend(getFactuality(c_src, c_gen, base_url=METRICS_URL, timeout=900.0))

    # readability (solo sobre generados)
    fkgl_all, smog_all, dale_all = [], [], []
    for c_gen in chunked(pairs_gen, BATCH_METRICS):
        rd = getReadability(c_gen, base_url=METRICS_URL, timeout=180.0)
        fkgl_all.extend(rd["fkgl"]); smog_all.extend(rd["smog"]); dale_all.extend(rd["dale_chall"])

    # registrar filas
    for sidx in range(len(gens)):
        rows.append({
            "doc_id": doc_id,
            "sample_idx": sidx,
            "relevance": float(rel_scores[sidx]),
            "factuality": float(fac_scores[sidx]),
            "fkgl": float(fkgl_all[sidx]),
            "smog": float(smog_all[sidx]),
            "dale_chall": float(dale_all[sidx]),
        })

per_sample_df = pd.DataFrame(rows).sort_values(["doc_id","sample_idx"]).reset_index(drop=True)
per_sample_df.head(), per_sample_df.shape

## Resultados

In [ ]:
def summarize_series(x: pd.Series) -> Dict[str, float]:
    v = x.dropna().to_numpy()
    if v.size == 0:
        return {"mean": None,"std": None,"min": None,"max": None,"p25": None,"p50": None,"p75": None,"n": 0}
    return {
        "mean": float(np.mean(v)),
        "std":  float(np.std(v, ddof=1)) if v.size > 1 else 0.0,
        "min":  float(np.min(v)),
        "max":  float(np.max(v)),
        "p25":  float(np.percentile(v, 25)),
        "p50":  float(np.percentile(v, 50)),
        "p75":  float(np.percentile(v, 75)),
        "n":    int(v.size),
    }

# Agregación global
stats = {
    "relevance":   summarize_series(per_sample_df["relevance"]),
    "factuality":  summarize_series(per_sample_df["factuality"]),
    "fkgl":        summarize_series(per_sample_df["fkgl"]),
    "smog":        summarize_series(per_sample_df["smog"]),
    "dale_chall":  summarize_series(per_sample_df["dale_chall"]),
}

# Imprimir estadísticas
print("=== Estadísticas globales ===")
for k, s in stats.items():
    print(f"\n{k.upper()}: n={s['n']}")
    print(f"  mean={s['mean']:.4f}  std={s['std']:.4f}  min={s['min']:.4f}  p25={s['p25']:.4f}  p50={s['p50']:.4f}  p75={s['p75']:.4f}  max={s['max']:.4f}")

# Graficar (una figura por métrica)
def plot_distribution(series: pd.Series, title: str):
    v = series.dropna().to_numpy()
    if v.size == 0:
        print(f"[skip] {title}: sin datos")
        return
    plt.figure(figsize=(7,4))
    # histograma simple
    plt.hist(v, bins=30, alpha=0.7)
    plt.title(title)
    plt.xlabel("valor")
    plt.ylabel("frecuencia")
    plt.grid(alpha=0.3)
    plt.show()

    # boxplot
    plt.figure(figsize=(5,4))
    plt.boxplot(v, vert=True, showmeans=True)
    plt.title(f"{title} — boxplot")
    plt.ylabel("valor")
    plt.grid(alpha=0.3)
    plt.show()

plot_distribution(per_sample_df["relevance"],  "Relevance")
plot_distribution(per_sample_df["factuality"], "Factuality")
plot_distribution(per_sample_df["fkgl"],       "FKGL")
plot_distribution(per_sample_df["smog"],       "SMOG")
plot_distribution(per_sample_df["dale_chall"], "Dale-Chall")

## Experimento en Databricks

In [ ]:
import os, json, time
import mlflow
from datetime import datetime
from mlflow.data import from_pandas


os.environ["DATABRICKS_HOST"] = KEYS.DATABRICKS_HOST
os.environ["DATABRICKS_TOKEN"] = KEYS.DATABRICKS_TOKEN
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/GeneracionDeResumenes/Estadísticas_de_Modelos")

# --- Preparación de artefactos a guardar ---
RUN_OUT = Path(OUT_DIR) / "mlflow_artifacts"
PLOTS_DIR = RUN_OUT / "plots"
RUN_OUT.mkdir(parents=True, exist_ok=True)
PLOTS_DIR.mkdir(parents=True, exist_ok=True)


# Graficar y guardar (hist + boxplot) para cada métrica
def save_distribution(series: pd.Series, title: str, slug: str):
    v = series.dropna().to_numpy()
    if v.size == 0:
        return
    plt.figure(figsize=(7,4))
    plt.hist(v, bins=30, alpha=0.7)
    plt.title(title); plt.xlabel("valor"); plt.ylabel("frecuencia"); plt.grid(alpha=0.3)
    hist_path = PLOTS_DIR / f"{slug}_hist.png"
    plt.savefig(hist_path, bbox_inches="tight"); plt.close()

    plt.figure(figsize=(5,4))
    plt.boxplot(v, vert=True, showmeans=True)
    plt.title(f"{title} — boxplot"); plt.ylabel("valor"); plt.grid(alpha=0.3)
    box_path = PLOTS_DIR / f"{slug}_box.png"
    plt.savefig(box_path, bbox_inches="tight"); plt.close()

save_distribution(per_sample_df["relevance"],  "Relevance",   "relevance")
save_distribution(per_sample_df["factuality"], "Factuality",  "factuality")
save_distribution(per_sample_df["fkgl"],       "FKGL",        "fkgl")
save_distribution(per_sample_df["smog"],       "SMOG",        "smog")
save_distribution(per_sample_df["dale_chall"], "Dale-Chall",  "dale_chall")

# --- Run name  ---
SAFE_MODEL_NAME = str(MODEL_DIR).replace(os.sep, "__").replace("/", "__").replace(" ", "_")
run_name = f"{SAFE_MODEL_NAME}-eval-{datetime.now().strftime('%Y%m%d_%H%M%S')}"


# --- Iniciar y loggear en MLflow ---
mlflow.end_run()
with mlflow.start_run(run_name=run_name):
    # Dataset de entrada (solo split usado en esta evaluación)
    try:
        ds_val = from_pandas(val_df, source="PLS_pairs", name=f"PLS_{SPLIT}")
        mlflow.log_input(ds_val, context="validation")
    except Exception as e:
        print("[MLflow] Aviso: no pude registrar dataset de validación:", e)

    # Parámetros de evaluación y generación
    mlflow.log_params({
        "model_dir": SAFE_MODEL_NAME,
        "device": DEVICE,
        "split": SPLIT,
        "num_examples": NUM_EXAMPLES,
        "samples_per_example": SAMPLES_PER_EXAMPLE,
        "System Prompt" : SYSTEM_PROMPT,
        "User_Prefix" : USER_PREFIX,
        **{f"gen_{k}": v for k, v in GEN_CFG.items()}, # parámetros de generación
    })

    # Métricas agregadas (prefijos por métrica)
    def log_agg(prefix: str, d: dict):
        clean = {k: float(v) for k, v in d.items() if isinstance(v, (int, float))}
        mlflow.log_metrics({f"{prefix}_{k}": v for k, v in clean.items()})
        if "n" in d:
            mlflow.log_metric(f"{prefix}_n", int(d["n"]))
    for m in ("relevance","factuality","fkgl","smog","dale_chall"):
        log_agg(m, stats[m])

    # Artefactos (CSV + stats + plots + config del adapter si existe)
    mlflow.log_artifact(str(per_sample_csv), artifact_path="metrics")
    mlflow.log_artifact(str(stats_json_path), artifact_path="metrics")
    for p in PLOTS_DIR.glob("*.png"):
        mlflow.log_artifact(str(p), artifact_path="plots")

    adapter_cfg = Path(MODEL_DIR) / "adapter_config.json"
    if adapter_cfg.exists():
        mlflow.log_artifact(str(adapter_cfg), artifact_path="model_info")

    # Marca de tiempo de fin
    mlflow.set_tag("finished_at", datetime.utcnow().isoformat() + "Z")

print("MLflow: evaluación registrada. Tracking URI:", mlflow.get_tracking_uri())